In [1]:
import pickle
from pathlib import Path
from textwrap import wrap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from scipy import stats
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
## USGS rain gage data

usgs_data_path = Path("usgs/usgs_Cook County.pkl")

with open(usgs_data_path, "rb") as f:
    data = pickle.load(f)

daily, insta, site_info, pcodes = data.values()

site_info.head()

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no,geometry
0,USGS,05530990,"SALT CREEK AT ROLLING MEADOWS, IL",ST,420337.41,880059.97,42.060392,-88.016658,N,5,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-88.01666 42.06039)
1,USGS,05536290,"LITTLE CALUMET RIVER AT SOUTH HOLLAND, IL",ST,413625.30,873551.30,41.607028,-87.597583,X,F,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.59758 41.60703)
2,USGS,413104087440001,"RAIN GAGE AT MATTESON, IL",AT,413104.00,874400.00,41.517778,-87.733333,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS0,POINT (-87.73333 41.51778)
3,USGS,413113087342201,"RAIN GAGE NEAR CHICAGO HEIGHTS, IL",AT,413115.00,873525.00,41.520868,-87.590321,M,S,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,00100,POINT (-87.59032 41.52087)
4,USGS,413115087352501,"RAIN GAGE AT DEER CREEK NEAR CHICAGO HEIGHTS, IL",AT,413115.00,873525.00,41.520833,-87.590278,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAWS00,POINT (-87.59028 41.52083)


In [3]:
pcodes

,parameter_cd,group,parm_nm,epa_equivalence,result_statistical_basis,result_time_basis,result_weight_basis,result_particle_size_basis,result_sample_fraction,result_temperature_basis,CASRN,SRSName,parm_unit
0,00065,Physical,"Gage height, feet",Agree,NaN,NaN,NaN,NaN,NaN,NaN,,"Height, gage",ft
0,00020,Physical,"Temperature, air, degrees Celsius",Agree,NaN,NaN,NaN,NaN,NaN,NaN,,"Temperature, air, deg C",deg C
0,00060,Physical,"Discharge, cubic feet per second",Not checked,Mean,1 Day,NaN,NaN,NaN,NaN,,"Stream flow, mean. daily",ft3/s
0,00045,Physical,"Precipitation, total, inches",Agree,NaN,NaN,NaN,NaN,NaN,NaN,,Precipitation,in


In [4]:
## Radar point query data
radar_pfiles = Path("./timeseries").glob("**/*.parquet")
radar_pfiles = sorted(radar_pfiles)

In [5]:
start = pd.Timestamp("2021-08-24T12:00:00", tz="UTC")
end = pd.Timestamp("2021-08-27T00:00:00", tz="UTC")

with PdfPages("timeseries/a_storm_comparison.pdf") as pdf:
    for i, radar_pfile in enumerate(radar_pfiles):
        station_name = radar_pfile.stem
        station_id = site_info.set_index("station_nm").loc[station_name]["site_no"]

        usgs_rain = insta.xs(station_id).loc[start:end, "00045"]

        if len(usgs_rain) == 0:
            continue

        tdelta_usgs = np.diff(usgs_rain.index)[0]
        usgs_rain_mmhr = usgs_rain * 25.4 / (tdelta_usgs / pd.Timedelta("1h"))  # from in to mm/hr

        radar_rain = pd.read_parquet(radar_pfile)
        radar_rain.index = pd.to_datetime(radar_rain.index, utc=True)
        radar_rain = radar_rain.loc[start:end]

        fig, axs = plt.subplots(
            3, 1, sharex=True, sharey=True, gridspec_kw={"hspace": 0.1}, figsize=(12, 6)
        )

        ax = axs[0]
        ax.set_title("MRMS Radar data", fontsize=8, va="top", y=0.9)
        ax.bar(radar_rain.index, radar_rain["value"], width=pd.Timedelta("1.9min"))

        ax = axs[1]
        homogenized = radar_rain.resample("2min").mean().fillna(0)
        downsampled = homogenized.resample("1min").interpolate("linear")
        resampled = downsampled.resample(tdelta_usgs).mean()
        ax.set_title(
            f"Resampled 2min -> {tdelta_usgs.seconds / 60}min", fontsize=8, va="top", y=0.9
        )
        ax.bar(resampled.index, resampled["value"], width=0.95 * tdelta_usgs)

        ax = axs[2]
        ax.set_title(f"USGS Rain gage\n{station_name}", fontsize=8, va="top", y=0.9)
        ax.bar(usgs_rain.index, usgs_rain_mmhr, width=0.95 * tdelta_usgs)

        for ax in axs:
            # ax.set_xlim(start, end)
            ax.xaxis.set_major_locator(mdates.AutoDateLocator())
            ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
            ax.spines.top.set_visible(False)
            ax.spines.right.set_visible(False)

        fig.supylabel("Precip Intensity [mm/h]", fontsize=10, fontweight=300, x=0.05)
        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)


> There is a gap in PALATINE between 2021/08/10 and 2021/09/08

In [6]:
start = pd.Timestamp("2021-08-24T00:00:00", tz="UTC")
end = pd.Timestamp("2021-08-30T00:00:00", tz="UTC")

with PdfPages("timeseries/a_storm_resampled_30min.pdf") as pdf:
    for radar_pfile in radar_pfiles:
        station_name = radar_pfile.stem
        station_id = site_info.set_index("station_nm").loc[station_name]["site_no"]

        usgs_rain = insta.xs(station_id).loc[start:end, "00045"]

        if len(usgs_rain) == 0:
            print(f"There is no data for {station_name} between {start} and {end}... Skipping")
            continue

        tdelta_usgs = np.diff(usgs_rain.index)[0]
        usgs_rain_mmhr = usgs_rain * 25.4 / (tdelta_usgs / pd.Timedelta("1h"))  # from in to mm/hr
        usgs_downsample = usgs_rain_mmhr.resample("1min").interpolate("linear")
        usgs_resample = usgs_downsample.resample(pd.Timedelta("30min")).mean()

        radar_rain = pd.read_parquet(radar_pfile)
        radar_rain.index = pd.to_datetime(radar_rain.index, utc=True)
        radar_rain = radar_rain.loc[start:end]
        radar_homogenized = radar_rain.resample("2min").mean().fillna(0)
        radar_downsample = radar_homogenized.resample("1min").interpolate("linear")
        radar_resample = radar_downsample.resample(pd.Timedelta("30min")).mean()

        fig, axs = plt.subplots(
            2, 1, sharex=True, sharey=True, gridspec_kw={"hspace": 0.1}, figsize=(12, 6)
        )

        ax = axs[0]
        ax.set_title("MRMS Radar (30min resample)", fontsize=8, va="top", y=0.9)
        ax.bar(radar_resample.index, radar_resample["value"], width=pd.Timedelta("29min"))

        ax = axs[1]
        ax.set_title("USGS Gauge (30min resample)", fontsize=8, va="top", y=0.9)
        ax.bar(usgs_resample.index, usgs_resample, width=pd.Timedelta("29min"))

        for ax in axs:
            ax.set_xlim(start, end)
            ax.xaxis.set_major_locator(mdates.AutoDateLocator())
            ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
            ax.spines.top.set_visible(False)
            ax.spines.right.set_visible(False)

        fig.supylabel("Precip Intensity [mm/h]", fontsize=10, fontweight=300, x=0.08)
        fig.suptitle(station_name, fontsize=10, y=0.92)
        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

There is no data for RAIN GAGE AT HARPER COLLEGE AT PALATINE, IL between 2021-08-24 00:00:00+00:00 and 2021-08-30 00:00:00+00:00... Skipping


In [ ]:
## We only have radar timeseries processed for 8/2021 here
from matplotlib.ticker import SymmetricalLogLocator

symlocator = SymmetricalLogLocator(base=10, linthresh=1, subs=range(10))

start = pd.Timestamp("2021-08-01T00:00:00", tz="UTC")
end = pd.Timestamp("2021-08-30T00:00:00", tz="UTC")

with PdfPages("timeseries/correlations.pdf") as pdf:
    for radar_pfile in radar_pfiles:
        station_name = radar_pfile.stem
        station_id = site_info.set_index("station_nm").loc[station_name]["site_no"]
        usgs_rain = insta.xs(station_id).loc[start:end, "00045"]

        if len(usgs_rain) == 0:
            print(f"There is no data for {station_name}... Skipping")
            continue

        tdelta_usgs = np.diff(usgs_rain.index)[0]
        usgs_rain_mmhr = usgs_rain * 25.4 / (tdelta_usgs / pd.Timedelta("1h"))  # from in to mm/hr
        usgs_downsample = usgs_rain_mmhr.resample("1min").interpolate("linear")
        usgs_resample = usgs_downsample.resample(pd.Timedelta("30min")).mean()

        radar_rain = pd.read_parquet(radar_pfile)
        radar_rain.index = pd.to_datetime(radar_rain.index, utc=True)
        radar_rain = radar_rain.loc[start:end]
        radar_homogenized = radar_rain.resample("2min").mean().fillna(0)
        radar_downsample = radar_homogenized.resample("1min").interpolate("linear")
        radar_resample = radar_downsample.resample(pd.Timedelta("30min")).mean()

        merged = pd.concat([radar_resample, usgs_resample], axis=1)
        merged = merged.dropna(axis="index")
        merged = merged.drop(merged[(merged["00045"] == 0) & (merged["value"] == 0)].index)

        # Pearson correlation
        rcorr = merged.corr()
        rsquared = np.power(rcorr.to_numpy(), 2)

        spearman = stats.spearmanr(merged["value"], merged["00045"])
        spearman = spearman.statistic

        fig, ax = plt.subplots()
        ax.set_title(f"{station_name}", fontsize=8, va="top", y=0.9)
        ax.text(
            0.98,
            0.02,
            f"$n={len(merged)}$\n$R^2={rsquared[0, 1]:.3f}$\n$\\rho={spearman:.3f}$",
            transform=ax.transAxes,
            ha="right",
        )
        ax.set_aspect("equal")
        ax.scatter(merged["value"], merged["00045"], marker=".", s=50, zorder=2)
        ax.set_xlabel("USGS 30-min resampled")
        ax.set_ylabel("MRMS 30-min resampled")
        ax.axline((0, 0), slope=1, ls="dotted", c="k")

        maxv = merged.max().max() * 1.2

        ax.set_xlim(-0.2, maxv)
        ax.set_ylim(-0.2, maxv)
        ax.set_xscale("symlog")
        ax.set_yscale("symlog")
        ax.xaxis.set_minor_locator(symlocator)
        ax.xaxis.grid(True, which="both", color="0.8", zorder=0)
        ax.yaxis.set_minor_locator(symlocator)
        ax.yaxis.grid(True, which="both", color="0.8", zorder=0)

        fig.tight_layout()
        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

In [8]:
## We only have radar timeseries processed for 8/2021 here
start = pd.Timestamp("2021-08-01T00:00:00", tz="UTC")
end = pd.Timestamp("2021-08-30T00:00:00", tz="UTC")
tres = pd.Timedelta("30min")

fig, ax = plt.subplots(figsize=(12, 7))

with PdfPages("timeseries/all_points_at_once.pdf") as pdf:
    for i, radar_pfile in enumerate(radar_pfiles):
        station_name = radar_pfile.stem
        station_id = site_info.set_index("station_nm").loc[station_name]["site_no"]

        usgs_rain = insta.xs(station_id).loc[start:end, "00045"]

        if len(usgs_rain) == 0:
            print(f"There is no data for {station_name}... Skipping")
            continue

        tdelta_usgs = np.diff(usgs_rain.index)[0]
        usgs_rain_mmhr = usgs_rain * 25.4 / (tdelta_usgs / pd.Timedelta("1h"))  # from in to mm/hr
        usgs_downsample = usgs_rain_mmhr.resample("1min").interpolate("linear")
        usgs_resample = usgs_downsample.resample(tres).mean()

        radar_rain = pd.read_parquet(radar_pfile)
        radar_rain.index = pd.to_datetime(radar_rain.index, utc=True)
        radar_rain = radar_rain.loc[start:end]
        radar_homogenized = radar_rain.resample("2min").mean().fillna(0)
        radar_downsample = radar_homogenized.resample("1min").interpolate("linear")
        radar_resample = radar_downsample.resample(tres).mean()

        merged = pd.concat([radar_resample, usgs_resample], axis=1)
        merged = merged.dropna(axis="index")
        merged = merged.drop(merged[(merged["00045"] == 0) & (merged["value"] == 0)].index)

        if i == 0:
            melted = merged.to_numpy()
        else:
            melted = np.append(melted, merged.to_numpy(), axis=0)

        ax.scatter(
            merged["value"],
            merged["00045"],
            marker=".",
            s=30,
            label="\n".join(wrap(station_name, 20)),
        )

    ax.set_xlabel(f"USGS {tres.seconds / 60:.0f}min resampled")
    ax.set_ylabel(f"MRMS {tres.seconds / 60:.0f}min resampled")
    ax.axline((0, 0), slope=1, ls="dotted", c="k")
    ax.set_aspect("equal")
    # ax.set_xlim(-1, 80)
    # ax.set_ylim(-1, 80)
    ax.set_xscale("symlog")
    ax.set_yscale("symlog")
    ax.legend(loc="center left", bbox_to_anchor=(1.0, 0.5), prop={"size": 6}, ncol=2)
    fig.tight_layout()
    pdf.savefig(fig, bbox_inches="tight")
    plt.close(fig)


In [9]:
## Stats on all data
rcorr = np.corrcoef(melted[:, 0], melted[:, 1])
rsquared = np.power(rcorr, 2)
spearman = stats.spearmanr(melted[:, 0], melted[:, 1])

print(f"Pearson's r = {rcorr[0, 1]:.3f}")
print(f"R² = {rsquared[0, 1]:.3f}")
print(f"Spearman ρ = {spearman.statistic:.3f}")

Pearson's r = 0.594
R² = 0.353
Spearman ρ = 0.675
